# Out-of-DIstribution detector for Neural networks (ODIN) with mobilenet

The goal of this notebook is to understand [Enhancing The Reliability of Out-of-distribution Image Detection in Neural Networks](https://arxiv.org/abs/1706.02690). We transform a trained classifier into a out-of-distribution detector. Here, we will take a small mobilenet neural network trained on CIFAR10 and see how to use it to make a detector for out-of-distirbution samples obtianed by cropping images from the ImageNet dataset.

The image below is taken from the original paper and shows the performances obtained with the ODIN method on DenseNet-BC-100 network.
![](https://raw.githubusercontent.com/ShiyuLiang/odin-pytorch/master/figures/original_optimal_shade.png)

A detector is a binary classifier thath needs to output $1$ when the input image is from the in-distribution (CIFAR10) and $0$ when the input image is from the out-distribution (ImageNet-crop).
TPR on CIFAR10 is then the probability that the detector is correct on an image from CIFAR10 and FPR on TinyImagenNet (crop) is the probability that the detector is not correct on an image from ImageNet-crop.
As in binary classification, we aim at maximizing TPR while minimizing FPR and the tradeoff between these two goals is capured by a ROC curve (shown above).

In this practical, we derive ROC curves for the ODIN methods with the mobilenet network.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import sys
import os

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Mobilenet for CIFAR10

We will use a pretrained model called [Mobilenet v2](https://pytorch.org/hub/pytorch_vision_mobilenet_v2/) which is an efficient network optimized for speed and memory, with [residual blocks](https://dataflowr.github.io/website/modules/17-resnets/). The default version from `torch.vision` is pretrained on Imagenet so we will rely on the version provided by [PyTorch_CIFAR10](https://github.com/huyvnphan/PyTorch_CIFAR10) which was trained on CIFAR10.

The code below allows you to download the weights of the neural network and the [piece of code](https://github.com/dataflowr/notebooks/blob/master/Module17/mobilenetv2.py) that will be useful for us (without cloning the whole repo above).

In [ ]:
# the code below downloads the weights of the neural net and should be run only once.
# You can uncomment it and run it, the first time you are using it or if you are running on colab.
#%mkdir state_dicts
#%cd state_dicts/
#!wget https://www.di.ens.fr/~lelarge/mobilenet_v2.pt
#%cd ..

In [ ]:
# if on colab first uncomment and run the following command (only once):
#!wget https://raw.githubusercontent.com/dataflowr/notebooks/master/Module17/mobilenetv2.py

In [ ]:
from mobilenetv2 import mobilenet_v2

model = mobilenet_v2(pretrained=True)
model.eval()
model.to(DEVICE)

In [ ]:
NORM_BIAS = [125.3 / 255, 123.0 / 255, 113.9 / 255]
NORM_SCALE = [63.0 / 255, 62.1 / 255.0, 66.7 / 255.0]
EPS_FSGM = 1e-2
IMAGE_SIZE = (32, 32)
batch_size = 64

%mkdir data

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(NORM_BIAS, NORM_SCALE),
        transforms.Resize(IMAGE_SIZE),
    ]
)

testset = torchvision.datasets.CIFAR10(root="./data/", train=False, download=True, transform=transform)
testloaderIn = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

## Question 1: test your network

Use the code from the [course](https://dataflowr.github.io/website/modules/5-stacking-layers/) (when we did overfit a MLP on CIFAR10) in order to get the performance of the pretrained network on CIFAR10 test set.

The dataloader is given to you above.

In [ ]:
# your code

## Out-of-distribution Dataset

The code below allows you to download the out-of-distribution dataset.
You should not modify this code.

In [ ]:
def imshow(inp, title=None):
#   Imshow for Tensor.
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(NORM_SCALE * inp + NORM_BIAS, 0,1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [ ]:
batchIn = next(iter(testloaderIn))

In [ ]:
imgIn = batchIn[0]
imgIn = torch.narrow(imgIn, 0, 0, 32)

In [ ]:
im_in = torchvision.utils.make_grid(imgIn)

imshow(im_in,title='images from CIFAR10')

In [ ]:
CIFAR10_labels = ['airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']
preds = model(imgIn.to(DEVICE)).max(1, keepdim=True)[1]
[CIFAR10_labels[pred] for pred in preds]

In [ ]:
# The code below dowload the out-of-distribution dataset
# if on colab first uncomment and run the following command (only once):
#%cd data
#!wget https://www.dropbox.com/s/avgm2u562itwpkl/Imagenet.tar.gz
#!tar -xvzf Imagenet.tar.gz
#%cd ..

In [ ]:
testsetout = torchvision.datasets.ImageFolder("./data/Imagenet/", transform=transform)
testloaderOut = DataLoader(testsetout, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
batchOut = next(iter(testloaderOut))

In [ ]:
imgOut = batchOut[0]
imgOut = torch.narrow(imgOut, 0, 0, 32)

In [ ]:
im_out = torchvision.utils.make_grid(imgOut)

imshow(im_out, title='images from ImageNet-crop')

We have now two dataloaders `testloaderIn` and `testloaderOut` corresponding to the in- and out-distributio, and ou mobilenet network `model`. 

# Out-of-distribution Detector

Our network `model` has been trained on CIFAR10 and hence outputs a vetor of size 10 representing the logprobs of being one of the 10 classes of CIFAR10. To use our model as a detector, we need to transform this output into a single value corresponding to the probability of being from the in-distribution. Hence, we ant thsi output to be high for images from CIFAR10 and low for images form ImageNet-crop.

## Temperature scaling

The first method proposed in [ODIN](https://arxiv.org/abs/1706.02690) relies on the fact that the trained network should be more confident on images from the in-distribution than on images form the out-distribution.

For an image ${\bf x}$, let $f({\bf x}) = (f_1({\bf x}),\dots, f_C({\bf x}))\in \mathbb{R}^C$ be the output of the network, where $C$ is the number of classes. We define for a temperature scaling parameter $T>0$,
\begin{eqnarray*}
S_i({\bf x},T) = \frac{\exp(f_i({\bf x})/T)}{\sum_{j=1}^C \exp(f_j({\bf x})/T)}\geq 0,
\end{eqnarray*}
so that $\sum_i S_i({\bf x},T)=1$. The **softmax score** is then $\max_i S_i({\bf x},T)$.

We expect the softmax score to be higher for images from the in-distribution than for images from the out-distribution. Hence we can build a classifier based on this score.

## Question 2: compute the softmax score

Compute the softmax scores with $T=1$ and $T=1000$ and plot the corresponding ROC curves (using [`metrics.roc_curve`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)).

Hint: to get a picture similar to the one in the paper shown above use `plt.ylim(0.8, 1.002)`.

In [ ]:
# this function takes a model, an algorithm and a dataloader and compute all the corresponding scores
# To use this function your algorithm needs to have a method apply taking as input 
# a batch and the model in order to compute the scores for the batch
def compute_scores(model, algo, loader, device=DEVICE):
    model.eval()
    all_scores = []
    for i, (batch, targets) in enumerate(loader):
        bs = batch.shape[0]
        batch = batch.to(device)
        scores = algo.apply(batch, model)
        all_scores += scores
    return all_scores
# You can test your method apply by running commands like:
# name_of_your_algo.apply(imgIn.to(DEVICE), model)
# name_of_your_algo.apply(imgOut.to(DEVICE), model)
# to obtain the scores on the small batchs made above.

In [ ]:
# your code

## Input preprocessing

The second method relies on perturbing the imput image. The main idea is to perturb the image in order to increase its softmax score as follows:
\begin{eqnarray*}
\tilde{\bf x} = {\bf x} - \epsilon \text{sign}\left( -\nabla_{\bf x} \log S_{\hat{y}}({\bf x},T)\right),
\end{eqnarray*}
where $\hat{y} =\arg\max_i S_i({\bf x},T)$ and $\epsilon>0$ is the noise magnitude applied on the image.
This perturbation should have a much higher impact on CIFAR10 images (in-distribution) than on ImageNet images (out-distribution) making the detection easier.

This second method can be used with the temperature scaling (this is why we keep the parameter $T$ in the formula above). Note that this method does not require the label of the image, as a result it can be applied to in-domain images as well as out-of domain images. The gradient can be computed with a standard [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) but note that the gradient is taken with respect to the pixels of the image not the weights of the neural network which are untouched. A similar technique has been used in [adversarial attacks](https://dataflowr.github.io/website/homework/2-CAM-adversarial/) where the goal was to perturb the image in order to fool the prediction made by the neural network.

## Question 3: code the perturbation on the image and the ODIN algorithm with both perturbation and temperature scaling.

Hints: 
- make a clone of the images and make it differentiable: 
```
inputs = images.clone()
inputs.requires_grad=True
```
- compute the label with `argmax`, the loss and do the backpropagation.
- compute the sign of the gradient with [`torch.ge`](https://pytorch.org/docs/stable/generated/torch.ge.html#torch-ge) for example to get your binary gradient
- remember that the images are first scaled with `NORM_SCALE`, hence divide your binary gradient by these quantity in order to implement exactly the algorithm described above (and to get better results!).
- for the ODIN algorithm creat a class as you did above with a `apply` method so that you can reuse your code for computing scores. You can test your algo with commands like
```
scores_In_odin = odin.apply(imgIn.to(DEVICE), model)
scores_Out_odin = odin.apply(imgOut.to(DEVICE), model)
```

Plot the resulting ROC curves.

In [ ]:
# your code